In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

# Enable mixed precision to reduce memory usage
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Constants with reduced requirements
IMG_SIZE = (256, 256)  # Reduced from 256x256
BATCH_SIZE = 64        # Reduced from 32
EPOCHS = 30
DATASET_PATH = './'
MODEL_PATH = 'saved_models'
TEST_VIDEO = 'test_video.mp4'

class MemoryEfficientDataset:
    def __init__(self):
        self.frame_paths = []
        self.labels = []
        
    def build_frame_dataset(self, max_samples_per_class=5000):
        """Build dataset by saving frames to disk to avoid memory overload"""
        os.makedirs('temp_frames/real', exist_ok=True)
        os.makedirs('temp_frames/fake', exist_ok=True)
        
        # Process real videos
        real_video_dir = os.path.join(DATASET_PATH, 'Celeb-real')
        self._process_videos(real_video_dir, 'real', max_samples_per_class)
        
        # Process fake videos
        fake_video_dir = os.path.join(DATASET_PATH, 'Celeb-synthesis')
        self._process_videos(fake_video_dir, 'fake', max_samples_per_class)
        
        # Balance classes
        min_samples = min(
            len([x for x in self.labels if x == 0]),
            len([x for x in self.labels if x == 1]))
        self.frame_paths = self.frame_paths[:min_samples*2]
        self.labels = self.labels[:min_samples*2]
        
    def _process_videos(self, video_dir, label_str, max_samples):
        """Process videos and save frames to disk"""
        label = 0 if label_str == 'real' else 1
        video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')][:max_samples//10]
        
        for video_file in tqdm(video_files, desc=f'Processing {label_str} videos'):
            video_path = os.path.join(video_dir, video_file)
            frame_count = 0
            
            cap = cv2.VideoCapture(video_path)
            while cap.isOpened() and frame_count < 10:  # Max 10 frames per video
                ret, frame = cap.read()
                if not ret:
                    break
                    
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, IMG_SIZE)
                
                # Save frame to disk
                frame_path = f'temp_frames/{label_str}/{video_file}_frame{frame_count}.jpg'
                cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
                
                self.frame_paths.append(frame_path)
                self.labels.append(label)
                frame_count += 1
                
            cap.release()
            
    def get_generator(self, batch_size=16):
        """Create memory-efficient data generator"""
        # Split paths and labels
        train_paths, test_paths, train_labels, test_labels = train_test_split(
            self.frame_paths, self.labels, test_size=0.2, random_state=42)
        
        # Custom generator to load images on demand
        def path_generator(paths, labels):
            while True:
                for i in range(0, len(paths), batch_size):
                    batch_paths = paths[i:i+batch_size]
                    batch_labels = labels[i:i+batch_size]
                    
                    batch_images = []
                    for path in batch_paths:
                        img = cv2.imread(path)
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = (img / 127.5) - 1.0  # Normalize to [-1, 1]
                        batch_images.append(img)
                    
                    yield np.array(batch_images, dtype=np.float32), np.array(batch_labels, dtype=np.float32)
        
        train_steps = len(train_paths) // batch_size
        test_steps = len(test_paths) // batch_size
        
        return path_generator(train_paths, train_labels), train_steps, \
               path_generator(test_paths, test_labels), test_steps

class LiteDeepfakeDetector:
    def __init__(self):
        self.discriminator = self.build_lite_discriminator()
        self.generator = self.build_lite_generator()
        self.gan = self.build_lite_gan()
        
    def build_lite_generator(self):
        """Memory-efficient generator with depthwise convolutions"""
        inputs = layers.Input(shape=(*IMG_SIZE, 3))
        
        # Downsample
        x = layers.Conv2D(32, (4,4), strides=2, padding='same')(inputs)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.SeparableConv2D(64, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.SeparableConv2D(128, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        # Bottleneck
        x = layers.SeparableConv2D(256, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        # Upsample
        x = layers.Conv2DTranspose(128, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        
        x = layers.Conv2DTranspose(64, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        
        x = layers.Conv2DTranspose(32, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        
        outputs = layers.Conv2DTranspose(3, (4,4), strides=2, padding='same', 
                                       activation='tanh')(x)
        
        return models.Model(inputs, outputs)
    
    def build_lite_discriminator(self):
        """Memory-efficient discriminator with separable convolutions"""
        inputs = layers.Input(shape=(*IMG_SIZE, 3))
        
        x = layers.SeparableConv2D(32, (4,4), strides=2, padding='same')(inputs)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.SeparableConv2D(64, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.SeparableConv2D(128, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.SeparableConv2D(256, (4,4), strides=1, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        # PatchGAN output
        patch_output = layers.Conv2D(1, (4,4), padding='same')(x)
        
        # Global pooling for classification
        y = layers.GlobalAveragePooling2D()(x)
        y = layers.Dense(1, activation='sigmoid')(y)
        
        return models.Model(inputs, [patch_output, y])
    
    def build_lite_gan(self):
        """Lite GAN model"""
        self.discriminator.trainable = False
        img_input = layers.Input(shape=(*IMG_SIZE, 3))
        generated_img = self.generator(img_input)
        gan_output, class_output = self.discriminator(generated_img)
        return models.Model(img_input, [gan_output, class_output])
    
    def compile_models(self, lr=0.0002):
        """Compile models with memory optimizations"""
        opt = tf.keras.optimizers.Adam(lr, beta_1=0.5)
        
        self.discriminator.compile(
            optimizer=opt,
            loss=['mse', 'binary_crossentropy'],
            loss_weights=[0.5, 0.5]
        )
        
        self.generator.compile(
            optimizer=opt,
            loss='mae'
        )
        
        self.gan.compile(
            optimizer=opt,
            loss=['mse', 'binary_crossentropy']
        )
    
    def train(self, train_gen, train_steps, test_gen, test_steps, epochs=EPOCHS):
        """Memory-efficient training"""
        real_labels = np.ones((BATCH_SIZE, *self.discriminator.output_shape[0][1:]))
        fake_labels = np.zeros((BATCH_SIZE, *self.discriminator.output_shape[0][1:]))
        
        detection_real = np.zeros((BATCH_SIZE, 1))
        detection_fake = np.ones((BATCH_SIZE, 1))
        
        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")
            
            # Train discriminator
            d_losses = []
            for _ in tqdm(range(train_steps), desc="Training discriminator"):
                real_imgs, _ = next(train_gen)
                
                # Generate fake images
                noise = tf.random.normal(real_imgs.shape)
                fake_imgs = self.generator.predict(noise, verbose=0)
                
                # Train on real images
                d_loss_real = self.discriminator.train_on_batch(
                    real_imgs, [real_labels[:len(real_imgs)], detection_real[:len(real_imgs)]])
                
                # Train on fake images
                d_loss_fake = self.discriminator.train_on_batch(
                    fake_imgs, [fake_labels[:len(fake_imgs)], detection_fake[:len(fake_imgs)]])
                
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                d_losses.append(d_loss)
            
            # Train generator
            g_losses = []
            for _ in tqdm(range(train_steps), desc="Training generator"):
                real_imgs, _ = next(train_gen)
                noise = tf.random.normal(real_imgs.shape)
                g_loss = self.gan.train_on_batch(
                    noise, [real_labels[:len(real_imgs)], detection_fake[:len(real_imgs)]])
                g_losses.append(g_loss)
            
            # Evaluate
            avg_d_loss = np.mean(d_losses, axis=0)
            avg_g_loss = np.mean(g_losses, axis=0)
            val_acc = self.evaluate(test_gen, test_steps)
            
            print(f"Discriminator Loss: {avg_d_loss[0]:.4f} | Generator Loss: {avg_g_loss[0]:.4f}")
            print(f"Validation Accuracy: {val_acc:.4f}")
    
    def evaluate(self, test_gen, test_steps):
        """Memory-efficient evaluation"""
        correct = 0
        total = 0
        
        for _ in range(test_steps):
            imgs, labels = next(test_gen)
            _, preds = self.discriminator.predict(imgs, verbose=0)
            preds = (preds > 0.5).astype(int)
            correct += np.sum(preds.flatten() == labels)
            total += len(labels)
            
        return correct / total
    
    def save_models(self):
        """Save models with reduced precision"""
        os.makedirs(MODEL_PATH, exist_ok=True)
        
        # Save weights only to reduce file size
        self.generator.save_weights(os.path.join(MODEL_PATH, 'generator_weights.h5'))
        self.discriminator.save_weights(os.path.join(MODEL_PATH, 'discriminator_weights.h5'))
    
    def load_models(self):
        """Load models with architecture recreation"""
        self.generator = self.build_lite_generator()
        self.discriminator = self.build_lite_discriminator()
        self.gan = self.build_lite_gan()
        
        self.generator.load_weights(os.path.join(MODEL_PATH, 'generator_weights.h5'))
        self.discriminator.load_weights(os.path.join(MODEL_PATH, 'discriminator_weights.h5'))

class EfficientVideoTester:
    def __init__(self, detector):
        self.detector = detector
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    def process_frame(self, frame):
        """Process single frame with memory efficiency"""
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) > 0:
            x, y, w, h = faces[0]
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, IMG_SIZE)
            face = (face / 127.5) - 1.0  # Normalize in-place
            return face
        return None
    
    def test_video(self, video_path, threshold=0.7, frame_interval=5):
        """Test video with memory-efficient frame processing"""
        cap = cv2.VideoCapture(video_path)
        frame_results = []
        frame_count = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            if frame_count % frame_interval == 0:  # Process every nth frame
                face = self.process_frame(frame)
                if face is not None:
                    face = np.expand_dims(face, axis=0)
                    _, detection_prob = self.detector.discriminator.predict(face, verbose=0)
                    frame_results.append(detection_prob[0][0])
            
            frame_count += 1
            
        cap.release()
        
        if not frame_results:
            print("No faces detected in video")
            return False, 0.0
        
        avg_prob = np.mean(frame_results)
        is_fake = avg_prob > threshold
        confidence = avg_prob if is_fake else 1 - avg_prob
        
        print("\nMemory-Efficient Test Results:")
        print(f"Frames processed: {len(frame_results)}/{frame_count}")
        print(f"Average score: {avg_prob:.4f}")
        print(f"Conclusion: {'FAKE' if is_fake else 'REAL'} (confidence: {confidence*100:.2f}%)")
        
        return is_fake, confidence

def main():
    # Initialize dataset
    print("Building memory-efficient dataset...")
    dataset = MemoryEfficientDataset()
    dataset.build_frame_dataset(max_samples_per_class=3000)  # Reduced sample count
    
    # Get generators
    train_gen, train_steps, test_gen, test_steps = dataset.get_generator(BATCH_SIZE)
    
    # Initialize and train model
    detector = LiteDeepfakeDetector()
    detector.compile_models()
    
    print("\nTraining memory-efficient model...")
    detector.train(train_gen, train_steps, test_gen, test_steps)
    detector.save_models()
    
    # Test video
    print("\nTesting video with memory optimizations...")
    tester = EfficientVideoTester(detector)
    is_fake, confidence = tester.test_video(TEST_VIDEO)
    
    print("\nFinal Result:")
    print(f"Video '{TEST_VIDEO}' is classified as: {'FAKE' if is_fake else 'REAL'}")
    print(f"Confidence: {confidence*100:.2f}%")

if __name__ == "__main__":
    # Clear session to free memory
    tf.keras.backend.clear_session()
    main()

In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

# Enable mixed precision to reduce memory usage while maintaining model size
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Constants - updated to new frame size
IMG_SIZE = (500, 942)  # Updated to 942×500 (height, width)
BATCH_SIZE = 16        # Reduced batch size further to accommodate larger images
EPOCHS = 50            # Increased epochs for better training
DATASET_PATH = './'
MODEL_PATH = 'saved_models'
TEST_VIDEO = 'test_video.mp4'

class MemoryEfficientDataset:
    def __init__(self):
        self.frame_paths = []
        self.labels = []
        
    def build_frame_dataset(self, max_samples_per_class=5000):
        """Build dataset by saving frames to disk to avoid memory overload"""
        os.makedirs('temp_frames/real', exist_ok=True)
        os.makedirs('temp_frames/fake', exist_ok=True)
        
        # Process real videos
        real_video_dir = os.path.join(DATASET_PATH, 'Celeb-real')
        self._process_videos(real_video_dir, 'real', max_samples_per_class)
        
        # Process fake videos
        fake_video_dir = os.path.join(DATASET_PATH, 'Celeb-synthesis')
        self._process_videos(fake_video_dir, 'fake', max_samples_per_class)
        
        # Balance classes
        min_samples = min(
            len([x for x in self.labels if x == 0]),
            len([x for x in self.labels if x == 1]))
        self.frame_paths = self.frame_paths[:min_samples*2]
        self.labels = self.labels[:min_samples*2]
        
    def _process_videos(self, video_dir, label_str, max_samples):
        """Process videos and save frames to disk"""
        label = 0 if label_str == 'real' else 1
        video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')][:max_samples//10]
        
        for video_file in tqdm(video_files, desc=f'Processing {label_str} videos'):
            video_path = os.path.join(video_dir, video_file)
            frame_count = 0
            
            cap = cv2.VideoCapture(video_path)
            while cap.isOpened() and frame_count < 10:  # Max 10 frames per video
                ret, frame = cap.read()
                if not ret:
                    break
                    
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (IMG_SIZE[1], IMG_SIZE[0]))  # Updated to new size (width, height)
                
                # Save frame to disk
                frame_path = f'temp_frames/{label_str}/{video_file}_frame{frame_count}.jpg'
                cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
                
                self.frame_paths.append(frame_path)
                self.labels.append(label)
                frame_count += 1
                
            cap.release()
            
    def get_generator(self, batch_size=16):
        """Create memory-efficient data generator"""
        # Split paths and labels
        train_paths, test_paths, train_labels, test_labels = train_test_split(
            self.frame_paths, self.labels, test_size=0.2, random_state=42)
        
        # Add data augmentation
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            zoom_range=0.1,
            preprocessing_function=lambda x: (x / 127.5) - 1.0  # Normalize to [-1, 1]
        )
        
        # Custom generator to load images on demand with augmentation
        def path_generator(paths, labels):
            while True:
                # Shuffle indices for each epoch
                indices = np.arange(len(paths))
                np.random.shuffle(indices)
                shuffled_paths = [paths[i] for i in indices]
                shuffled_labels = [labels[i] for i in indices]
                
                for i in range(0, len(shuffled_paths), batch_size):
                    batch_paths = shuffled_paths[i:i+batch_size]
                    batch_labels = shuffled_labels[i:i+batch_size]
                    
                    batch_images = []
                    for path in batch_paths:
                        img = cv2.imread(path)
                        if img is None:
                            print(f"Warning: Could not read image {path}")
                            continue
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        batch_images.append(img)
                    
                    if not batch_images:
                        continue
                    
                    # Apply augmentation
                    batch_images = next(datagen.flow(
                        np.array(batch_images, dtype=np.float32),
                        shuffle=False,
                        batch_size=len(batch_images)))
                    
                    yield batch_images, np.array(batch_labels, dtype=np.float32)
        
        train_steps = max(1, len(train_paths) // batch_size)
        test_steps = max(1, len(test_paths) // batch_size)
        
        return path_generator(train_paths, train_labels), train_steps, \
               path_generator(test_paths, test_labels), test_steps

class EnhancedDeepfakeDetector:
    def __init__(self):
        # First create the discriminator standalone model - this is what we'll train
        self.discriminator = self.build_enhanced_discriminator()
        
        # The GAN model will only be used for inference so we build it last
        self.discriminator.summary()
    
    def build_enhanced_discriminator(self):
        """Enhanced discriminator with more capacity and attention"""
        inputs = layers.Input(shape=(*IMG_SIZE, 3))
        
        # Enhanced feature extraction with adjusted strides for larger images
        x = layers.Conv2D(64, (5,5), strides=3, padding='same')(inputs)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.Conv2D(128, (5,5), strides=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.Conv2D(256, (5,5), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        x = layers.Conv2D(512, (4,4), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        # Attention mechanism
        attention = layers.Conv2D(512, (1,1), activation='sigmoid')(x)
        x = layers.multiply([x, attention])
        
        # Additional convolutional layers for better feature extraction
        x = layers.Conv2D(512, (4,4), strides=1, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        
        # Enhanced classification head
        y = layers.GlobalAveragePooling2D()(x)
        y = layers.Dense(128, activation='relu')(y)
        y = layers.Dropout(0.3)(y)
        class_output = layers.Dense(1, activation='sigmoid', name='classification')(y)
        
        return models.Model(inputs, class_output)
    
    def compile_model(self, lr=0.0001):
        """Compile the discriminator model with appropriate optimizer"""
        opt = tf.keras.optimizers.Adam(lr, beta_1=0.5)
        
        self.discriminator.compile(
            optimizer=opt,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    
    def train(self, train_gen, train_steps, test_gen, test_steps, epochs=EPOCHS):
        """Enhanced training with callbacks and learning rate scheduling"""
        # Learning rate scheduler
        lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(
            lambda epoch, lr: lr * 0.9 if epoch > 0 and epoch % 5 == 0 else lr
        )
        
        # Early stopping callback
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True
        )
        
        # ModelCheckpoint to save best model
        os.makedirs(MODEL_PATH, exist_ok=True)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            os.path.join(MODEL_PATH, 'discriminator.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
        
        # TensorBoard logging
        tensorboard = tf.keras.callbacks.TensorBoard(
            log_dir='logs',
            write_graph=True,
            update_freq='epoch'
        )
        
        history = self.discriminator.fit(
            train_gen,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=test_gen,
            validation_steps=test_steps,
            callbacks=[lr_scheduler_callback, early_stopping, checkpoint, tensorboard]
        )
        
        # Plot training history
        self.plot_training_history(history)
        
        return history
    
    def plot_training_history(self, history):
        """Plot training history"""
        plt.figure(figsize=(12, 5))
        
        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.close()
    
    def evaluate(self, test_gen, test_steps):
        """Evaluate the model"""
        metrics = self.discriminator.evaluate(test_gen, steps=test_steps)
        print(f"Test Loss: {metrics[0]:.4f}, Test Accuracy: {metrics[1]:.4f}")
        return metrics[1]  # Return accuracy
    
    def save_model(self):
        """Save the model"""
        os.makedirs(MODEL_PATH, exist_ok=True)
        self.discriminator.save(os.path.join(MODEL_PATH, 'discriminator.h5'))
    
    def load_model(self):
        """Load saved model"""
        self.discriminator = tf.keras.models.load_model(os.path.join(MODEL_PATH, 'discriminator.h5'))

class EfficientVideoTester:
    def __init__(self, detector):
        self.detector = detector
    
    def process_frame(self, frame):
        """Process single frame with memory efficiency"""
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (IMG_SIZE[1], IMG_SIZE[0]))  # Updated to new size
        frame = (frame / 127.5) - 1.0  # Normalize in-place
        return frame
    
    def test_video(self, video_path, threshold=0.5, frame_interval=5):
        """Test video with memory-efficient frame processing"""
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                return False, 0.0
                
            frame_results = []
            frame_count = 0
            processed_count = 0
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                    
                if frame_count % frame_interval == 0:  # Process every nth frame
                    processed_frame = self.process_frame(frame)
                    processed_frame = np.expand_dims(processed_frame, axis=0)
                    detection_prob = self.detector.discriminator.predict(processed_frame, verbose=0)
                    frame_results.append(detection_prob[0])
                    processed_count += 1
                
                frame_count += 1
                
            cap.release()
            
            if not frame_results:
                print("No frames processed in video")
                return False, 0.0
            
            # Visualization of frame-by-frame predictions
            plt.figure(figsize=(12, 5))
            plt.plot(frame_results)
            plt.axhline(y=threshold, color='r', linestyle='--')
            plt.title('Frame-by-Frame Prediction Scores')
            plt.xlabel('Processed Frame')
            plt.ylabel('Deepfake Score')
            plt.ylim(0, 1)
            plt.savefig('frame_predictions.png')
            plt.close()
            
            avg_prob = np.mean(frame_results)
            is_fake = avg_prob > threshold
            confidence = avg_prob if is_fake else 1 - avg_prob
            
            print("\nMemory-Efficient Test Results:")
            print(f"Frames processed: {processed_count}/{frame_count}")
            print(f"Average score: {avg_prob:.4f}")
            print(f"Conclusion: {'FAKE' if is_fake else 'REAL'} (confidence: {confidence*100:.2f}%)")
            
            return is_fake, confidence
            
        except Exception as e:
            print(f"Error while testing video: {str(e)}")
            return False, 0.0

def main():
    print("Building enhanced dataset...")
    dataset = MemoryEfficientDataset()
    dataset.build_frame_dataset(max_samples_per_class=5000)  # Increased sample count
    
    # Get generators with augmentation
    train_gen, train_steps, test_gen, test_steps = dataset.get_generator(BATCH_SIZE)
    
    # Initialize and train enhanced model
    detector = EnhancedDeepfakeDetector()
    detector.compile_model()
    
    print("\nTraining enhanced model...")
    detector.train(train_gen, train_steps, test_gen, test_steps, epochs=EPOCHS)
    
    # Test video
    print("\nTesting video with enhanced model...")
    tester = EfficientVideoTester(detector)
    is_fake, confidence = tester.test_video(TEST_VIDEO)
    
    print("\nFinal Result:")
    print(f"Video '{TEST_VIDEO}' is classified as: {'FAKE' if is_fake else 'REAL'}")
    print(f"Confidence: {confidence*100:.2f}%")

if __name__ == "__main__":
    # Clear session to free memory
    tf.keras.backend.clear_session()
    main()

Building enhanced dataset...


Processing fake videos: 100%|██████████| 500/500 [00:20<00:00, 23.82it/s]
C:\Users\araut1\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(

C:\Users\araut1\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500, 942,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 167, 314,  │      4,864 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 167, 314,  │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 105,   │    204,928 │ leaky_re_lu[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 56, 105,   │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 56, 105,   │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 53,    │    819,456 │ leaky_re_lu_1[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 53,    │      1,024 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 28, 53,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 14, 27,    │  2,097,664 │ leaky_re_lu_2[0]… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 27,    │      2,048 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 14, 27,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 14, 27,    │    262,656 │ leaky_re_lu_3[0]… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 14, 27,    │          0 │ leaky_re_lu_3[0]… │
│                     │ 512)              │            │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 14, 27,    │  4,194,816 │ multiply[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 27,    │      2,048 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 14, 27,    │          0 │ batch_normalizat

 Total params: 7,655,809 (29.20 MB)

...

499/499 ━━━━━━━━━━━━━━━━━━━━ 2925s 6s/step - accuracy: 0.8869 - loss: 0.2927 - val_accuracy: 0.9036 - val_loss: 0.2782 - learning_rate: 8.1000e-05
Epoch 13/50
Epoch 13/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8988 - loss: 0.2719
Epoch 13: val_accuracy did not improve from 0.90358

Epoch 13: val_accuracy did not improve from 0.90358
499/499 ━━━━━━━━━━━━━━━━━━━━ 2936s 6s/step - accuracy: 0.8988 - loss: 0.2719 - val_accuracy: 0.8975 - val_loss: 0.2808 - learning_rate: 8.1000e-05
Epoch 14/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 2936s 6s/step - accuracy: 0.8988 - loss: 0.2719 - val_accuracy: 0.8975 - val_loss: 0.2808 - learning_rate: 8.1000e-05
Epoch 14/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8879 - loss: 0.2872
Epoch 14: val_accuracy improved from 0.90358 to 0.91065, saving model to saved_models\discriminator.h5

Epoch 14: val_accuracy improved from 0.90358 to 0.91065, saving model to saved_models\discriminator.h5


499/499 ━━━━━━━━━━━━━━━━━━━━ 2925s 6s/step - accuracy: 0.8879 - loss: 0.2871 - val_accuracy: 0.9107 - val_loss: 0.2698 - learning_rate: 8.1000e-05
Epoch 15/50
Epoch 15/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9013 - loss: 0.2745
Epoch 15: val_accuracy did not improve from 0.91065
499/499 ━━━━━━━━━━━━━━━━━━━━ 2954s 6s/step - accuracy: 0.9013 - loss: 0.2745 - val_accuracy: 0.8713 - val_loss: 0.3160 - learning_rate: 8.1000e-05

Epoch 15: val_accuracy did not improve from 0.91065
499/499 ━━━━━━━━━━━━━━━━━━━━ 2954s 6s/step - accuracy: 0.9013 - loss: 0.2745 - val_accuracy: 0.8713 - val_loss: 0.3160 - learning_rate: 8.1000e-05
Epoch 16/50
Epoch 16/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9063 - loss: 0.2665
Epoch 16: val_accuracy did not improve from 0.91065
499/499 ━━━━━━━━━━━━━━━━━━━━ 2955s 6s/step - accuracy: 0.9063 - loss: 0.2665 - val_accuracy: 0.8718 - val_loss: 0.3246 - learning_rate: 7.2900e-05
Epoch 17/50

Epoch 16: val_accuracy did not improve from 0.

499/499 ━━━━━━━━━━━━━━━━━━━━ 2974s 6s/step - accuracy: 0.9017 - loss: 0.2630 - val_accuracy: 0.9122 - val_loss: 0.2721 - learning_rate: 7.2900e-05
Epoch 20/50
Epoch 20/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9114 - loss: 0.2447
Epoch 20: val_accuracy did not improve from 0.91217
499/499 ━━━━━━━━━━━━━━━━━━━━ 2968s 6s/step - accuracy: 0.9114 - loss: 0.2447 - val_accuracy: 0.8809 - val_loss: 0.3055 - learning_rate: 7.2900e-05

Epoch 20: val_accuracy did not improve from 0.91217
499/499 ━━━━━━━━━━━━━━━━━━━━ 2968s 6s/step - accuracy: 0.9114 - loss: 0.2447 - val_accuracy: 0.8809 - val_loss: 0.3055 - learning_rate: 7.2900e-05
Epoch 21/50
Epoch 21/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9137 - loss: 0.2403
Epoch 21: val_accuracy did not improve from 0.91217
499/499 ━━━━━━━━━━━━━━━━━━━━ 2987s 6s/step - accuracy: 0.9136 - loss: 0.2403 - val_accuracy: 0.9076 - val_loss: 0.2604 - learning_rate: 6.5610e-05

Epoch 21: val_accuracy did not improve from 0.91217
499/49

499/499 ━━━━━━━━━━━━━━━━━━━━ 720s 1s/step - accuracy: 0.9135 - loss: 0.2485 - val_accuracy: 0.9182 - val_loss: 0.2346 - learning_rate: 6.5610e-05
Epoch 24/50
Epoch 24/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9148 - loss: 0.2434
Epoch 24: val_accuracy did not improve from 0.91822
499/499 ━━━━━━━━━━━━━━━━━━━━ 710s 1s/step - accuracy: 0.9148 - loss: 0.2434 - val_accuracy: 0.8995 - val_loss: 0.2728 - learning_rate: 6.5610e-05
Epoch 25/50

Epoch 24: val_accuracy did not improve from 0.91822
499/499 ━━━━━━━━━━━━━━━━━━━━ 710s 1s/step - accuracy: 0.9148 - loss: 0.2434 - val_accuracy: 0.8995 - val_loss: 0.2728 - learning_rate: 6.5610e-05
Epoch 25/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9169 - loss: 0.2344
Epoch 25: val_accuracy did not improve from 0.91822
499/499 ━━━━━━━━━━━━━━━━━━━━ 698s 1s/step - accuracy: 0.9169 - loss: 0.2344 - val_accuracy: 0.9096 - val_loss: 0.2725 - learning_rate: 6.5610e-05
Epoch 26/50

Epoch 25: val_accuracy did not improve from 0.9182

499/499 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.9220 - loss: 0.2217 - val_accuracy: 0.9197 - val_loss: 0.2537 - learning_rate: 5.9049e-05
Epoch 29/50
Epoch 29/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9206 - loss: 0.2261
Epoch 29: val_accuracy did not improve from 0.91974
499/499 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.9206 - loss: 0.2261 - val_accuracy: 0.8945 - val_loss: 0.2691 - learning_rate: 5.9049e-05

Epoch 29: val_accuracy did not improve from 0.91974
499/499 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.9206 - loss: 0.2261 - val_accuracy: 0.8945 - val_loss: 0.2691 - learning_rate: 5.9049e-05
Epoch 30/50
Epoch 30/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9134 - loss: 0.2453
Epoch 30: val_accuracy did not improve from 0.91974
499/499 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.9134 - loss: 0.2452 - val_accuracy: 0.9107 - val_loss: 0.2503 - learning_rate: 5.9049e-05
Epoch 31/50

Epoch 30: val_accuracy did not improve from 0.9197

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

class VideoTester:
    def __init__(self, model_path='saved_models/discriminator.h5'):
        # Load the trained model
        self.model = tf.keras.models.load_model(model_path)
        self.img_size = (500, 942)  # Should match your training size
        
    def process_frame(self, frame):
        """Process a single frame for prediction"""
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (self.img_size[1], self.img_size[0]))
        frame = (frame / 127.5) - 1.0  # Normalize to [-1, 1]
        return np.expand_dims(frame, axis=0)
    
    def test_video(self, video_path, threshold=0.5, frame_interval=5):
        """Test a video file and return prediction results"""
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                return None
            
            frame_results = []
            frame_count = 0
            processed_count = 0
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                    
                if frame_count % frame_interval == 0:  # Process every nth frame
                    processed_frame = self.process_frame(frame)
                    prediction = self.model.predict(processed_frame, verbose=0)[0][0]
                    frame_results.append(prediction)
                    processed_count += 1
                
                frame_count += 1
                
            cap.release()
            
            if not frame_results:
                print(f"No frames processed in video {video_path}")
                return None
            
            avg_prob = np.mean(frame_results)
            is_fake = avg_prob > threshold
            confidence = avg_prob if is_fake else 1 - avg_prob
            
            return {
                'video_path': video_path,
                'is_fake': is_fake,
                'confidence': confidence,
                'avg_score': avg_prob,
                'frames_processed': f"{processed_count}/{frame_count}",
                'frame_predictions': frame_results
            }
            
        except Exception as e:
            print(f"Error while testing video {video_path}: {str(e)}")
            return None
    
    def test_videos_in_folder(self, folder_path, threshold=0.5, frame_interval=5):
        """Test all videos in a folder and return results"""
        if not os.path.exists(folder_path):
            print(f"Error: Folder {folder_path} does not exist")
            return []
            
        video_files = [f for f in os.listdir(folder_path) 
                      if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        
        if not video_files:
            print(f"No video files found in {folder_path}")
            return []
        
        results = []
        for video_file in tqdm(video_files, desc="Testing videos"):
            video_path = os.path.join(folder_path, video_file)
            result = self.test_video(video_path, threshold, frame_interval)
            if result is not None:
                results.append(result)
        
        return results
    
    def generate_report(self, results, output_file='test_results.txt'):
        """Generate a text report of the testing results"""
        with open(output_file, 'w') as f:
            f.write("Deepfake Video Test Results\n")
            f.write("="*50 + "\n\n")
            
            for result in results:
                f.write(f"Video: {result['video_path']}\n")
                f.write(f"Classification: {'FAKE' if result['is_fake'] else 'REAL'}\n")
                f.write(f"Confidence: {result['confidence']*100:.2f}%\n")
                f.write(f"Average Score: {result['avg_score']:.4f}\n")
                f.write(f"Frames Processed: {result['frames_processed']}\n")
                
                # Plot frame predictions
                plt.figure(figsize=(10, 4))
                plt.plot(result['frame_predictions'])
                plt.axhline(y=0.5, color='r', linestyle='--')
                plt.title(f"Frame Predictions - {os.path.basename(result['video_path'])}")
                plt.xlabel('Processed Frame')
                plt.ylabel('Deepfake Score')
                plt.ylim(0, 1)
                
                plot_filename = f"predictions_{os.path.splitext(os.path.basename(result['video_path']))[0]}.png"
                plt.savefig(plot_filename)
                plt.close()
                
                f.write(f"Prediction plot saved to: {plot_filename}\n")
                f.write("\n" + "-"*50 + "\n\n")
            
            # Summary statistics
            if results:
                fake_count = sum(1 for r in results if r['is_fake'])
                real_count = len(results) - fake_count
                avg_confidence = np.mean([r['confidence'] for r in results]) * 100
                
                f.write("\nSummary Statistics:\n")
                f.write(f"Total Videos Tested: {len(results)}\n")
                f.write(f"Fake Videos Detected: {fake_count}\n")
                f.write(f"Real Videos Detected: {real_count}\n")
                f.write(f"Average Confidence: {avg_confidence:.2f}%\n")
        
        print(f"\nReport generated and saved to {output_file}")

def test_all_videos():
    # Initialize the tester
    tester = VideoTester()
    
    # Path to your test videos folder
    test_folder = 'test_videos'
    
    # Test all videos in the folder
    results = tester.test_videos_in_folder(test_folder)
    
    # Generate a report
    if results:
        tester.generate_report(results)
    else:
        print("No valid results to report.")

if __name__ == "__main__":
    test_all_videos()

Building full-frame dataset with all frames...


Processing fake videos: 100%|██████████| 166/166 [03:47<00:00,  1.37s/it]



ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
